In [3]:
import os
import numpy as np
import pylab
from scipy.io import wavfile
from pathlib import Path

# Function to get sound and frame info using scipy
def get_wav_info(wav_file):
    frame_rate, sound_info = wavfile.read(wav_file)
    return sound_info, frame_rate

# Paths
input_directory = 'C:/Users/esruj/Desktop/Fall 2023/Capstone'  # Replace with your input folder path
output_directory = 'C:/Users/esruj/Desktop/Fall 2023/Capstone/Images' # Replace with your output folder path
os.makedirs(output_directory, exist_ok=True)

# Process and convert WAV to spectrogram PNG
for filename in os.listdir(input_directory):
    if filename.endswith('.wav'):
        file_path = os.path.join(input_directory, filename)
        file_stem = Path(file_path).stem
        file_dist_path = os.path.join(output_directory, file_stem)

        # Converting signal to spectrogram and saving as PNG
        if not os.path.exists(file_dist_path + '.png'):
            sound_info, frame_rate = get_wav_info(file_path)
            pylab.specgram(sound_info, Fs=frame_rate)
            pylab.savefig(f'{file_dist_path}.png')
            pylab.close()

print("Conversion completed!")


<ipython-input-3-d2fd581be269>:9: WavFileWarning: Chunk (non-data) not understood, skipping it.
  frame_rate, sound_info = wavfile.read(wav_file)


Conversion completed!


In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import spectrogram, butter, lfilter, medfilt

# Function to apply bandpass filter
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y

# Source folder containing .wav files
source_folder = 'C:/Users/esruj/Desktop/Fall 2023/Capstone'  # Replace with your source folder path

# Target folder to save spectrograms
target_folder = 'C:/Users/esruj/Desktop/Fall 2023/Capstone/Images1'  # Replace with your target folder path
os.makedirs(target_folder, exist_ok=True)  # Create target folder if it doesn't exist

# Process each .wav file in the source folder
for filename in os.listdir(source_folder):
    if filename.lower().endswith('.wav'):
        file_path = os.path.join(source_folder, filename)

        # Load PCG signal
        fs, signal = wavfile.read(file_path)  # fs is the sampling frequency

        # Noise reduction
        signal = medfilt(signal)  # Median filter
        signal = bandpass_filter(signal, 25, 400, fs)  # Bandpass filter (adjust frequencies as needed)

        # Generate spectrogram
        frequencies, times, Sxx = spectrogram(signal, fs)

        # Plotting the spectrogram
        plt.figure(figsize=(10, 4))
        plt.pcolormesh(times, frequencies, 10 * np.log10(Sxx), shading='gouraud')
        plt.ylabel('Frequency [Hz]')
        plt.xlabel('Time [sec]')
        plt.title(f'Spectrogram of {filename}')
        plt.colorbar(label='Intensity [dB]')

        # Save the spectrogram as an image
        spectrogram_path = os.path.join(target_folder, f'{os.path.splitext(filename)[0]}_spectrogram.png')
        plt.savefig(spectrogram_path)
        plt.close()  # Close the plot to free up memory

print("Processing complete. Spectrograms", target_folder)

Processing complete. Spectrograms C:/Users/esruj/Desktop/Fall 2023/Capstone/Images1


In [7]:
pip install PyWavelets

Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import spectrogram, medfilt
import pywt

# Function to apply wavelet denoising
def wavelet_denoise(signal, wavelet='db6', level=1):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    threshold = (np.sqrt(2*np.log(len(signal)))) / np.sqrt(2)
    new_coeffs = list(map(lambda x: pywt.threshold(x, threshold, mode='soft'), coeffs))
    denoised_signal = pywt.waverec(new_coeffs, wavelet)
    return denoised_signal

# Source folder containing .wav files
source_folder = 'C:/Users/esruj/Desktop/Fall 2023/Capstone'  # Replace with your source folder path

# Target folder to save spectrograms
target_folder = 'C:/Users/esruj/Desktop/Fall 2023/Capstone/Images1' 
os.makedirs(target_folder, exist_ok=True)  

# Process each .wav file in the source folder
for filename in os.listdir(source_folder):
    if filename.lower().endswith('.wav'):
        file_path = os.path.join(source_folder, filename)

        # Load PCG signal
        fs, signal = wavfile.read(file_path)  # fs is the sampling frequency

        # Noise reduction using wavelet denoising
        signal = medfilt(signal)  # Median filter
        signal = wavelet_denoise(signal)  # Wavelet denoising

        # Generate spectrogram
        frequencies, times, Sxx = spectrogram(signal, fs)

        # Plotting the spectrogram
        plt.figure(figsize=(10, 4))
        plt.pcolormesh(times, frequencies, 10 * np.log10(Sxx), shading='gouraud')
        plt.ylabel('Frequency [Hz]')
        plt.xlabel('Time [sec]')
        plt.title(f'Spectrogram of {filename}')
        plt.colorbar(label='Intensity [dB]')

        # Save the spectrogram as an image
        spectrogram_path = os.path.join(target_folder, f'{os.path.splitext(filename)[0]}_spectrogram.png')
        plt.savefig(spectrogram_path)
        plt.close()  # Close the plot to free up memory

print("Processing complete. Spectrograms saved to:", target_folder)

<ipython-input-8-538f2ace82d2>:29: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, signal = wavfile.read(file_path)  # fs is the sampling frequency
<ipython-input-8-538f2ace82d2>:40: RuntimeWarning: divide by zero encountered in log10
  plt.pcolormesh(times, frequencies, 10 * np.log10(Sxx), shading='gouraud')


Processing complete. Spectrograms saved to: C:/Users/esruj/Desktop/Fall 2023/Capstone/Images1
